### Imports

In [ ]:
import re
import requests
import io
import string
from PyPDF2 import PdfFileReader

### Get PDF text

In [ ]:
pdfurl = "http://arxiv.org/pdf/1811.04422v1"
r = requests.get(pdfurl, stream=True)
f = io.BytesIO(r.content)
reader = PdfFileReader(f)

# Get text
text = ""
page = 0
while True:
    try:
        content = reader.getPage(page).extractText()
        text += content
        page += 1
    except:
        break

### Extract abstract

In [ ]:
text = text.split("\n\n")

# collect abstract text
abstract = []
cnt = 0

if "\n \n" in " ".join(i for i in text):
    content = []
    for i in text:
        for j in i.split("\n \n"):
            content.append(re.sub("\n", "", j))
    while cnt < len(content):
        if "abstract" in content[cnt].lower():
            if "abstract" in content[cnt].lower() and len(text[cnt].split()) < 10:
                cnt += 1
            while True:
                if "introduction" in content[cnt].lower():
                    break
                abstract.append(content[cnt])
                cnt += 1
            break
        else:
            cnt += 1
else:
    while cnt < len(text):
        if "abstract" in text[cnt].lower():
            if "abstract" in text[cnt].lower() and len(text[cnt].split()) < 10:
                cnt += 1
            while True:
                if "*" in text[cnt] and len(text[cnt].split()) < 10:
                    cnt += 1
                elif len(text[cnt].split()) > 10:
                    if "introduction" in text[cnt][:20].lower() or "i ntroduction" in text[cnt][:20].lower():
                        break
                    abstract.append(text[cnt])
                    cnt +=1
                elif len(text[cnt].split()) == 1:
                    try:
                        int(text[cnt])
                    except:
                        break
                    break
                else:
                    break
            break
        else:
            cnt += 1
    if not abstract:
        stop = 0
        while True:
            if "introduction" in text[stop].lower() and len(text[stop].split()) < 5:
                break
            else:
                stop += 1
        cnt = 0
        while cnt < stop:
            if text[cnt] == " " or len(text[cnt]) < 2:
                if len(text[cnt+1].split()) < 10:
                    cnt += 1
                else:
                    abstract.append(text[cnt+1])
                    if text[cnt+1][-1] == "-" or text[cnt+1][:-1] == "-":
                        while True:
                            if text[cnt+2][0].islower():
                                abstract.append(text[cnt+2])
                                break
                            else:
                                cnt += 1
            cnt += 1
            
# format abstract to present to user
abstract_string = ""
for a in abstract:
    a = a.strip()
    a = re.sub("-\n", "", a)
    a = re.sub("\n", " ", a)
    if a[-1] not in string.punctuation:
        abstract_string += a+" "
    elif a[-1] == "-" or a[-1] == "-":
        abstract_string += a[:-1]
    elif a[-1] in string.punctuation:
        abstract_string += a+" "
# remove word 'abstract' and date at start
if "abstract" in abstract_string[:8].lower():
    abstract_string = abstract_string[8:]
dummy1 = abstract_string.split(". ")
dummy2 = abstract_string.split()
if "abstract" in dummy1[0].lower():
    print(dummy1[0])
    cnt = 0
    while True:
        if dummy2[cnt].lower().strip() != "abstract":
            cnt += 1
        else:
            cnt += 1
            break
    abstract_string = " ".join(word for word in dummy2[cnt:])
if abstract_string[:1] in string.punctuation or abstract_string[:1] == "—":
    abstract_string = abstract_string[1:]
abstract_string = abstract_string.strip()
print("Abstract:\n\n", abstract_string)

### Testing the function

In [1]:
from abstractapi import abstractextracter

In [3]:
abstractextracter("https://arxiv.org/pdf/2001.09950.pdf")

'We present a framework for deformable object manipulation that interleaves planning and control, enabling complex manipulation tasks without relying on high-fidelity modeling or simulation. The key question we address is when should we use planning and when should we use control to achieve the task? Planners are designed to find paths through complex configuration spaces, but for highly underactuated systems, such as deformable objects, achieving a specific configuration is very difficult even with high-fidelity models. Conversely, controllers can be designed to achieve specific configurations, but they can be trapped in undesirable local minima due to obstacles. Our approach consists of three components: (1) A global motion planner to generate gross motion of the deformable object; (2) A local controller for refinement of the configuration of the deformable object; and (3) A novel deadlock prediction algorithm to determine when to use planning versus control. By separating planning f